In [1]:
import pandas
import os
from sklearn.model_selection import train_test_split

In [4]:
dataframe = pandas.read_csv('..\\Data\\Processed Data\\Temperature.csv')

print(dataframe.shape)
print(dataframe)

(95694, 6)
                      date  temperature_one  temperature_two  \
0      1756-01-01 00:00:00             -8.0             -9.0   
1      1756-01-02 00:00:00             -9.0             -9.0   
2      1756-01-03 00:00:00            -10.0             -7.0   
3      1756-01-04 00:00:00             -7.0             -8.0   
4      1756-01-05 00:00:00             -9.0             -5.0   
...                    ...              ...              ...   
95689  2017-12-27 00:00:00              2.9              3.5   
95690  2017-12-28 00:00:00              4.3              4.9   
95691  2017-12-29 00:00:00              2.9              2.7   
95692  2017-12-30 00:00:00              1.4              1.4   
95693  2017-12-31 00:00:00              2.5              3.2   

       temperature_three  temperature_four  temperature_five  
0                    NaN               NaN               NaN  
1                    NaN               NaN               NaN  
2                    NaN       